In [ ]:
import os
import pandas as pd
from policies.mappo import MAPPO
from environment.optimization_environment import OptimizationEnv
from environment.utils import parse_config
from datetime import datetime
import numpy as np
import seaborn as sns
import os
import neptune
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy
from other_algorithms.pso import ParticleSwarmOptimizer
import time
from scipy.stats import multivariate_normal
from dotenv import load_dotenv
load_dotenv()

In [ ]:
def initialize(config_path, mode="train", **kwargs):
    env = OptimizationEnv(config_path)
    agent_policy = MAPPO(config_path)
    if mode == "test" or mode == "benchmark":
        model_path = kwargs.get("model_path", None)
        if model_path is None:
            raise ValueError("Model path must be provided for testing")
        agent_policy.load(model_path)
    return env, agent_policy

def print_items(**kwargs):
    for key, value in kwargs.items():
        print(key, value)
        
def get_action(observation_info, agent_policy, env):
    observation, observation_std = observation_info
    actions = np.zeros((env.n_agents, env.n_dim))
    for dim in range(env.n_dim):
        observation[dim] = observation[dim].astype(np.float32)
        #print(observation[dim])
        observation_std[dim] = observation_std[dim].astype(np.float32)
        action = agent_policy.select_action(observation[dim], observation_std[dim])
        actions[:, dim] = action
    return actions

# def cal_densities(points):
#     densities = np.array([multivariate_normal(mean=mean, cov=cov, allow_singular=True).pdf(points)
#                               for mean, cov in zip(env.gmm.gmm.means_, env.gmm.gmm.covariances_)])
#     return densities

# def cal_densities_variances(densities):
#     variances = []
#     for dim in range(densities.shape[0]):
#         densities_variances = np.var(densities[dim], axis=0)
#         print(densities_variances)
#         variances.append(densities_variances)
#     return variances

In [ ]:
config_path = 'config/config.json'
model_path = "models/model.pth"
mode = "train"
env, agent_policy = initialize(config_path, mode=mode, model_path=model_path)
config = parse_config(config_path)
agent_policy.set_action_std(config["test_action_std"])

def display_gmm_details(gmm):
    print("Means: ", gmm.means_)
    print("Covariances: ", gmm.covariances_)
    print("Weights: ", gmm.weights_)


In [ ]:
_ = env.reset()

In [ ]:
observation_info = env.reset()
rewards = []
std_comp = []
hig_std_comp = []
iters = 11
results_buffer = np.zeros((iters, 4))
for i in range(iters):
    print("Iteration: ", i)
    # if i % 1 == 0:
        # _, std = env.surrogate.evaluate(env.surrogate.checkpoints)
        # _ = env.surrogate.cal_reward(
        # std_comp.append(1-std_std)
        # print("Reward: ", reward)
        # print("Std: ", std_std)
        # print("total_reward: ", reward + (1 - std_std))
        # combined_reward = reward * np.exp(-0.1 * max(0, 1- std_std))
        # results_buffer[i] = [reward, (1 - std_std), reward * (1 - std_std), combined_reward]
        # # env.surrogate.plot_variance()
        
        # rewards.append(reward + (1 - std_std))
        
    #action =  get_action(observation_info, agent_policy, env)
    action = np.random.uniform(-0.3, 0.3, (env.n_agents, env.n_dim))
    observation_info, reward, done, info = env.step(action)
    rewards.append(np.mean(reward)) 
    env.surrogate.plot_checkpoints_state()
    print("Reward: ", np.mean(reward))
    print(1 - env.surrogate.percent_high_std / 100)
    std_std = np.std(env.surrogate.checkpoints_std)
    print(f"Std - {std_std}")
    
    

    

plt.plot(rewards)

In [ ]:
env.surrogate.plot_checkpoints_state()

In [ ]:
env.render(type="history", file_path="new_history.gif")

In [ ]:
fig = plt.figure(figsize=(10, 10), dpi=80)
plt.plot(results_buffer[:, 2], label="total reward")
plt.plot(results_buffer[:, 0], label="reward")
plt.plot(results_buffer[:, 1], label="std")

plt.legend()
plt.show()


In [ ]:
results_buffer

In [ ]:
mean, std = env.surrogate.evaluate(env.surrogate.checkpoints)
mean2, std2 = env.surrogate.evaluate(env.surrogate.checkpoints)

# check if the mean and std are the same
print(np.all(mean == mean2))
print(np.all(std == std2))